In [2]:
from pydub import AudioSegment
import os, traceback, sys
from tqdm import tqdm
import pandas as pd
import numpy as np

In [73]:
class Words_Seperator:
    def __init__(self, datacsv, segmentdatafileloc, savingloc, sentence_npy) -> None:
        self.datacsv = datacsv
        self.sgmtdatfileloc = segmentdatafileloc
        self.saveloc = savingloc
        self.sentence_npy = sentence_npy
        self.columns = []
        self.datadict = {}
        self.filedict = {}
        self.newdata = []

    def CorrectFileName(self,val):
        parts = val.split('.')[0].split('_')
        if(len(parts)>1):
            splitit = parts[3].split('-')
            if(len(splitit)==1): parts[3]=str(int(parts[3]))
            else: parts[3]=str(int(splitit[0]))+'-'+str(int(splitit[1]))
        return '_'.join(parts)

    def gendata(self):
        data = pd.read_csv(self.datacsv)
        self.columns = data.columns
        data = data.to_numpy()
        for i in data: self.datadict[self.CorrectFileName(i[0])]=np.array(i[1:])
    
    def genfile(self):
        try: files = list(filter(lambda x:x.split('.')[1]=='npy',os.listdir(self.sentence_npy)))
        except Exception as e:
            traceback.print_exc()
            sys.exit(3)
        for i in files: self.filedict[self.CorrectFileName(i)]=i

    def getsegments(self,filename):
        data = []
        with open(os.path.join(self.sgmtdatfileloc,filename),'r') as f:
            lines = f.readlines()
        for i in lines:
            try: 
                splited_line = i.split('\t')
                word = splited_line[0]
                start = float(splited_line[1])
                end = float(splited_line[2])
                data.append([start,end,word])
            except Exception as e: print(e)
        return data

    def run(self):
        self.genfile()
        self.gendata()
        try: segmentsfiles = list(filter(lambda x:x.split('.')[1]=='txt',os.listdir(self.sgmtdatfileloc)))
        except Exception as e:
            traceback.print_exc()
            sys.exit(3)
#         print(len(segmentsfiles))
#         count=0
        for i in tqdm(segmentsfiles):
#             print(self.CorrectFileName(i))
#             count+=1
            if(self.CorrectFileName(i) in self.filedict and self.CorrectFileName(i) in self.datadict):
                stamps = self.getsegments(i)
                fileloc = os.path.join(self.sentence_npy,self.filedict[self.CorrectFileName(i)])
                data = np.load(fileloc)
                word_count = {}
                length = data[0].shape[0]
                if(length==0 or len(stamps)==0): continue
                maxtime = stamps[-1][1]
                for current in stamps:
                    start,end,word=int((current[0]*length)/maxtime),int((current[1]*length)/maxtime),current[2]
                    if(word in word_count): word_count[word]+=1
                    else: word_count[word]=1
                    npysave = data[start:end]
                    with open(os.path.join(self.saveloc,f'{self.CorrectFileName(i)}_{word}_{word_count[word]}.npy'),'wb+') as f: 
                        np.save(f,npysave)
                    val = [f'{self.CorrectFileName(i)}_{word}_{word_count[word]}.npy']
                    val = val + list(self.datadict[self.CorrectFileName(i)])
                    self.newdata.append(val)
        df = pd.DataFrame(self.newdata)
#         print(count)
        df = df.set_axis(self.columns,axis=1)
        df.to_csv('newdata.csv',index=False)

if __name__ == '__main__':
    savingloc = './words_npy/'
    datacsv = './data.csv'
    sentence_npy = './student_resampled_wav/'
    segmentdatafileloc = './word_level_boundaries/'
    cinstance = Words_Seperator(datacsv,segmentdatafileloc,savingloc,sentence_npy)
    cinstance.run()


  2%|██▎                                                                                                                                                 | 410/26232 [00:11<11:45, 36.59it/s]


KeyboardInterrupt: 